In [5]:
import pandas as pd

In [8]:
test = pd.read_csv('/Users/t2023-m0149/Documents/spartacodingclub/Projects/4. insurance/test.csv')

In [9]:
sample = pd.read_csv('/Users/t2023-m0149/Documents/spartacodingclub/Projects/4. insurance/sample.csv')

In [12]:
train = pd.read_csv('/Users/t2023-m0149/Documents/spartacodingclub/Projects/4. insurance/train.csv')

In [13]:
print("전체 행 개수 :", len(train))
print(train.isna().sum())     

전체 행 개수 : 1200000
id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64


## 데이터 전처리(나머지 변수, 공통)

In [14]:
import numpy as np


# Number of Dependents
train['Number of Dependents'] = train['Number of Dependents'].astype('Int64')
train['Number of Dependents'] = train['Number of Dependents'].astype('category')
train['Number of Dependents'] = train['Number of Dependents'].cat.add_categories('Missing')
train['Number of Dependents'] = train['Number of Dependents'].fillna('Missing')


# Age
train['Age'] = train['Age'].fillna(train['Age'].median())


# Annual Income
train['Annual Income'] = train['Annual Income'].fillna(train['Annual Income'].median())


# Health Score
train['Health Score'] = train['Health Score'].fillna(train['Health Score'].median())


# Credit Score
train['Credit Score'] = train['Credit Score'].fillna('Missing')


# Customer Feedback
train['Customer Feedback'] = train['Customer Feedback'].fillna('No Feedback')


# Marital Status
train['Marital Status'] = train['Marital Status'].fillna('Unknown')


# Vehicle Age 결측 행 삭제
train = train[train['Vehicle Age'].notnull()]


# Insurance Duration 결측 행 삭제
train = train[train['Insurance Duration'].notnull()]

In [15]:
train.isna().sum()

id                           0
Age                          0
Gender                       0
Annual Income                0
Marital Status               0
Number of Dependents         0
Education Level              0
Occupation              358074
Health Score                 0
Location                     0
Policy Type                  0
Previous Claims         364028
Vehicle Age                  0
Credit Score                 0
Insurance Duration           0
Policy Start Date            0
Customer Feedback            0
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

### 상관계수

In [16]:
# pip install pandas numpy seaborn matplotlib scipy scikit-learn


In [17]:
# import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import scipy.stats as stats
# from sklearn.feature_selection import mutual_info_regression
# from sklearn.preprocessing import LabelEncoder


# # 전처리: Number_of_Dependents_bin 생성
# train['Number of Dependents'] = train['Number of Dependents'].fillna(0)
# def bin_dependents(x):
#    if x == 0:
#        return '0'
#    elif x <= 2:
#        return '1-2'
#    else:
#        return '3+'
# train['Number_of_Dependents_bin'] = train['Number of Dependents'].apply(bin_dependents)


# # 1) 수치형 간 Pearson 상관분석
# num_cols = ['Age', 'Annual Income', 'Health Score', 'Vehicle Age', 'Insurance Duration']
# corr = train[num_cols].corr(method='pearson')
# plt.figure(figsize=(8, 6))
# sns.heatmap(corr, annot=True, cmap='coolwarm', square=True)
# plt.title('📊 Pearson Correlation: Numeric Features')
# plt.show()


# # 2) 범주형-수치형 분석: Boxplot + ANOVA
# sns.boxplot(x='Marital Status', y='Age', data=train)
# plt.title('🎯 Age by Marital Status')
# plt.show()


# grouped = [train.loc[train['Marital Status'] == g, 'Age'].dropna() for g in train['Marital Status'].unique()]
# anova_result = stats.f_oneway(*grouped)
# print('ANOVA p-value:', anova_result.pvalue)


# # 3) 범주형-범주형 분석: 카이제곱 + Cramér’s V + Countplot
# contingency = pd.crosstab(train['Marital Status'], train['Number_of_Dependents_bin'])
# chi2, p, dof, expected = stats.chi2_contingency(contingency)
# print('Chi2-test p-value:', p)


# def cramers_v(conf_matrix):
#    chi2 = stats.chi2_contingency(conf_matrix)[0]
#    n = conf_matrix.sum().sum()
#    r, k = conf_matrix.shape
#    return np.sqrt(chi2 / (n * (min(r, k)-1)))


# print("Cramér’s V:", cramers_v(contingency))


# sns.countplot(data=train, x='Marital Status', hue='Number_of_Dependents_bin')
# plt.title('📦 Countplot of Marital Status by Dependents Bin')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


# # 4) Mutual Information 분석 (혼합형)
# df_enc = train.copy()


# # 범주형 변수 인코딩
# for col in df_enc.select_dtypes(include=['object', 'category']).columns:
#    df_enc[col] = LabelEncoder().fit_transform(df_enc[col].astype(str))


# # Y값 (target 변수) 설정
# y = df_enc['Premium Amount']  # ← target 변수명 필요 시 수정
# X = df_enc.drop(columns=['Premium Amount'])


# # Mutual Information 계산
# mi = mutual_info_regression(X, y, discrete_features='auto')
# mi_df = pd.DataFrame({'feature': X.columns, 'mutual_info': mi})
# mi_df = mi_df.sort_values(by='mutual_info', ascending=False)


# # 시각화
# plt.figure(figsize=(12, 6))
# sns.barplot(x='feature', y='mutual_info', data=mi_df)
# plt.title('🧠 Mutual Information with Target')
# plt.xticks(rotation=90)s
# plt.tight_layout()
# plt.show()

# 실험1

1. Occupation
- 1-1. 'unknown'으로 대체
- 1-2. 범주별 annual income값 비교해서 대체

2. Previous Claims
- 2-1. 0값으로 처리
- 2-2. 0~9 범주화, null로 대체

### 실험1 > 1-1 & 2-1

In [18]:
train1 = train.copy()
train1['Occupation'] = train1['Occupation'].fillna('Unknown')


train1['Previous Claims'] = train1['Previous Claims'].fillna(0)

In [11]:
train2 = train.copy()
train2['Occupation'] = train2['Occupation'].fillna('Unknown')


train2['Previous Claims'] = train2['Previous Claims'].astype('Int64')
train2['Previous Claims'] = train2['Previous Claims'].astype('category')
train2['Previous Claims'] = train2['Previous Claims'].cat.add_categories('Missing')
train2['Previous Claims'] = train2['Previous Claims'].fillna('Missing')

In [12]:
train3 = train.copy()
occupation_income_mean = train.groupby('Occupation')['Annual Income'].mean()
def fill_occupation(row):
  if pd.isnull(row['Occupation']):
      diffs = (occupation_income_mean - row['Annual Income']).abs()
      return diffs.idxmin()
  else:
      return row['Occupation']


train3['Occupation'] = train3.apply(fill_occupation, axis=1)


train3['Previous Claims'] = train3['Previous Claims'].fillna(0)

In [13]:
train4 = train.copy()
occupation_income_mean = train.groupby('Occupation')['Annual Income'].mean()
def fill_occupation(row):
  if pd.isnull(row['Occupation']):
      diffs = (occupation_income_mean - row['Annual Income']).abs()
      return diffs.idxmin()
  else:
      return row['Occupation']


train4['Occupation'] = train4.apply(fill_occupation, axis=1)


train4['Previous Claims'] = train4['Previous Claims'].astype('Int64')
train4['Previous Claims'] = train4['Previous Claims'].astype('category')
train4['Previous Claims'] = train4['Previous Claims'].cat.add_categories('Missing')
train4['Previous Claims'] = train4['Previous Claims'].fillna('Missing')

### CatBoost

In [14]:
pip install catboost


You should consider upgrading via the '/Users/t2023-m0149/Documents/GitHub/4gun4_new_project/alex/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# 2. 타겟 설정
target1 = 'Premium Amount'
y1 = train1[target1]
X1 = train1.drop(columns=[target1])


# 3. 결측치 처리 및 범주형 컬럼 지정
cat_features1 = X1.select_dtypes(include=['object', 'category']).columns.tolist()


# NaN을 문자열 'Missing'으로 변환 (CatBoost 호환)
for col in cat_features1:
   X1[col] = X1[col].astype(str).fillna("Missing")


# 숫자형 컬럼 NaN은 중앙값으로 대체
for col in X1.select_dtypes(include=[np.number]).columns:
   if X1[col].isnull().sum() > 0:
       X1[col] = X1[col].fillna(X1[col].median())


# 4. 학습/검증 데이터 분리
X_train1, X_val1, y_train1, y_val1 = train_test_split(X1, y1, test_size=0.2, random_state=42)


# 5. 모델 정의
model1 = CatBoostRegressor(
   iterations=1000,
   learning_rate=0.05,
   depth=6,
   eval_metric='MAE',
   cat_features=cat_features1,
   random_seed=42,
   verbose=100
)


# 6. 모델 학습
model1.fit(X_train1, y_train1, eval_set=(X_val1, y_val1), use_best_model=True)


# 7. 예측 및 평가
y_pred1 = model1.predict(X_val1)


# 다양한 평가지표 출력
mae = mean_absolute_error(y_val1, y_pred1)
mse = mean_squared_error(y_val1, y_pred1)
rmse = np.sqrt(mse)
r2 = r2_score(y_val1, y_pred1)


print(f"📌 MAE:  {mae:.4f}")
print(f"📌 MSE:  {mse:.4f}")
print(f"📌 RMSE: {rmse:.4f}")
print(f"📌 R2:   {r2:.4f}")ㄴ


SyntaxError: invalid syntax. Perhaps you forgot a comma? (713573081.py, line 63)

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score




target2 = 'Premium Amount'
y2 = train2[target2]
X2 = train2.drop(columns=[target2])




# 올바른 cat_features 추출 (train2 기준)
cat_features2 = X2.select_dtypes(include=['object', 'category']).columns.tolist()




# 범주형 결측 문자 'Missing' 대체
for col in cat_features2:
  X2[col] = X2[col].astype(str).fillna('Missing')




# 수치형 결측은 중앙값 대체
for col in X2.select_dtypes(include=[np.number]).columns:
  if X2[col].isnull().sum() > 0:
      X2[col] = X2[col].fillna(X2[col].median())




# cat_features 리스트와 실제 컬럼 타입 확인
print("Categorical Features:", cat_features2)
print(X2.dtypes[cat_features2])




# 학습/검증 데이터 분할
X_train2, X_val2, y_train2, y_val2 = train_test_split(X2, y2, test_size=0.2, random_state=42)




# 모델 정의
model2 = CatBoostRegressor(
  iterations=1000,
  learning_rate=0.05,
  depth=6,
  eval_metric='MAE',
  cat_features=cat_features2,
  random_seed=42,
  verbose=100
)




# 모델 학습
model2.fit(X_train2, y_train2, eval_set=(X_val2, y_val2), use_best_model=True)




# 예측 및 평가
y_pred2 = model2.predict(X_val2)
mae = mean_absolute_error(y_val2, y_pred2)
mse = mean_squared_error(y_val2, y_pred2)
rmse = np.sqrt(mse)
r2 = r2_score(y_val2, y_pred2)




print(f"train2 MAE:  {mae:.4f}")
print(f"train2 MSE:  {mse:.4f}")
print(f"train2 RMSE: {rmse:.4f}")
print(f"train2 R2:   {r2:.4f}")


Categorical Features: ['Gender', 'Marital Status', 'Number of Dependents', 'Education Level', 'Occupation', 'Location', 'Policy Type', 'Previous Claims', 'Credit Score', 'Policy Start Date', 'Customer Feedback', 'Smoking Status', 'Exercise Frequency', 'Property Type']
Gender                  object
Marital Status          object
Number of Dependents    object
Education Level         object
Occupation              object
Location                object
Policy Type             object
Previous Claims         object
Credit Score            object
Policy Start Date       object
Customer Feedback       object
Smoking Status          object
Exercise Frequency      object
Property Type           object
dtype: object
0:	learn: 667.9957958	test: 668.7504018	best: 668.7504018 (0)	total: 1.27s	remaining: 21m 10s
100:	learn: 653.7569083	test: 654.0719997	best: 654.0719997 (100)	total: 5m 39s	remaining: 50m 24s
200:	learn: 650.4555579	test: 650.7101920	best: 650.7101920 (200)	total: 14m 32s	remaining

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error




# 2. 타겟 설정
target3 = 'Premium Amount'
y3 = train3[target3]
X3 = train3.drop(columns=[target3])




# 3. 결측치 처리 및 범주형 컬럼 지정
cat_features3 = X3.select_dtypes(include=['object', 'category']).columns.tolist()




# NaN을 문자열 'Missing'으로 변환 (CatBoost 호환)
for col in cat_features3:
  X3[col] = X3[col].astype(str).fillna("Missing")




# 숫자형 컬럼 NaN은 중앙값으로 대체
for col in X3.select_dtypes(include=[np.number]).columns:
  if X3[col].isnull().sum() > 0:
      X3[col] = X3[col].fillna(X3[col].median())




# 4. 학습/검증 데이터 분리
X_train3, X_val3, y_train3, y_val3 = train_test_split(X3, y3, test_size=0.2, random_state=42)




# 5. 모델 정의
model3 = CatBoostRegressor(
  iterations=1000,
  learning_rate=0.05,
  depth=6,
  eval_metric='MAE',
  cat_features=cat_features3,
  random_seed=42,
  verbose=100
)




# 6. 모델 학습
model3.fit(X_train3, y_train3, eval_set=(X_val3, y_val3), use_best_model=True)




# 예측 및 평가
y_pred3 = model3.predict(X_val3)
mae = mean_absolute_error(y_val3, y_pred3)
mse = mean_squared_error(y_val3, y_pred3)
rmse = np.sqrt(mse)
r2 = r2_score(y_val2, y_pred3)




print(f"train3 MAE:  {mae:.4f}")
print(f"train3 MSE:  {mse:.4f}")
print(f"train3 RMSE: {rmse:.4f}")
print(f"train3 R2:   {r2:.4f}")






0:	learn: 667.9864127	test: 668.7337461	best: 668.7337461 (0)	total: 2.22s	remaining: 36m 55s
100:	learn: 653.4596191	test: 654.0788837	best: 654.0788837 (100)	total: 3m 37s	remaining: 32m 12s
200:	learn: 650.3516463	test: 650.8452309	best: 650.8452309 (200)	total: 6m 36s	remaining: 26m 16s
300:	learn: 647.9209334	test: 648.2849533	best: 648.2849533 (300)	total: 8m 42s	remaining: 20m 13s
400:	learn: 646.5360110	test: 647.0060495	best: 647.0060495 (400)	total: 9m 58s	remaining: 14m 53s
500:	learn: 645.8194568	test: 646.3612762	best: 646.3612762 (500)	total: 11m 13s	remaining: 11m 10s
600:	learn: 645.0868405	test: 645.7117993	best: 645.7101199 (589)	total: 12m 25s	remaining: 8m 15s
700:	learn: 644.6423115	test: 645.3490746	best: 645.3449007 (695)	total: 13m 41s	remaining: 5m 50s
800:	learn: 644.0361328	test: 644.8465411	best: 644.8465373 (788)	total: 14m 55s	remaining: 3m 42s
900:	learn: 643.7255767	test: 644.6738483	best: 644.6655824 (885)	total: 16m 7s	remaining: 1m 46s
999:	learn: 643

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# 2. 타겟 설정
target4 = 'Premium Amount'
y4 = train4[target4]
X4 = train4.drop(columns=[target4])


# 3. 결측치 처리 및 범주형 컬럼 지정
cat_features4 = X4.select_dtypes(include=['object', 'category']).columns.tolist()


# NaN을 문자열 'Missing'으로 변환 (CatBoost 호환)
for col in cat_features4:
   X4[col] = X4[col].astype(str).fillna("Missing")


# 숫자형 컬럼 NaN은 중앙값으로 대체
for col in X4.select_dtypes(include=[np.number]).columns:
   if X4[col].isnull().sum() > 0:
       X4[col] = X4[col].fillna(X4[col].median())


# 4. 학습/검증 데이터 분리
X_train4, X_val4, y_train4, y_val4 = train_test_split(X4, y4, test_size=0.2, random_state=42)


# 5. 모델 정의
model4 = CatBoostRegressor(
   iterations=1000,
   learning_rate=0.05,
   depth=6,
   eval_metric='MAE',
   cat_features=cat_features4,
   random_seed=42,
   verbose=100
)


# 6. 모델 학습
model4.fit(X_train4, y_train4, eval_set=(X_val4, y_val4), use_best_model=True)


# 예측 및 평가
y_pred4 = model3.predict(X_val4)
mae = mean_absolute_error(y_val4, y_pred4)
mse = mean_squared_error(y_val4, y_pred4)
rmse = np.sqrt(mse)
r2 = r2_score(y_val4, y_pred4)




print(f"train4 MAE:  {mae:.4f}")
print(f"train4 MSE:  {mse:.4f}")
print(f"train4 RMSE: {rmse:.4f}")
print(f"train4 R2:   {r2:.4f}")

0:	learn: 667.9957958	test: 668.7504018	best: 668.7504018 (0)	total: 693ms	remaining: 11m 31s
100:	learn: 654.1391893	test: 654.4613850	best: 654.4613850 (100)	total: 1m 14s	remaining: 11m
200:	learn: 650.1962425	test: 650.3591072	best: 650.3591072 (200)	total: 6m 10s	remaining: 24m 34s
300:	learn: 648.1702904	test: 648.2604486	best: 648.2604486 (300)	total: 11m 7s	remaining: 25m 49s
400:	learn: 646.6817101	test: 646.8220760	best: 646.8220760 (400)	total: 15m 56s	remaining: 23m 48s
